In [317]:
# Standard stack
import datetime
import pandas as pd
import numpy as np
from datetime import date
import re

# Visualization
from pandas_profiling import ProfileReport
#import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
import tensorflow as tf
import tensorflow_decision_forests as tfdf
try:
    from wurlitzer import sys_pipes
except:
    from colabtools.googlelog import CaptureLog as sys_pipes

# Scikit-learn packages
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold

# display
from IPython.core.magic import register_line_magic
from IPython.display import Javascript

# no warnings
import warnings
warnings.filterwarnings('ignore')

In [318]:
TRAIN_PATH = "data/train.csv"
TEST_PATH = "data/test.csv"
label = "outcome_type"

In [319]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
submission = pd.read_csv(TEST_PATH)

In [320]:
classes = train[label].unique().tolist()
print(f"Label classes: {classes}")

train[label] = train[label].map(classes.index)

Label classes: ['adoption', 'no outcome', 'transfer']


In [321]:
train.columns

Index(['id', 'age_upon_outcome', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'name', 'outcome_type', 'sex',
       'spay_neuter'],
      dtype='object')

In [322]:
test.columns

Index(['id', 'age_upon_outcome', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'name', 'sex', 'spay_neuter'],
      dtype='object')

# Train

In [323]:
def inpute_missing(dataset):
    """ 
    Edit this to fix nulls. Default version replaces all int/float with 0
    """
    for col in dataset.columns:
        if dataset[col].dtype not in [str, object]:
            dataset[col] = dataset[col].fillna(dataset[col].mean())
    return dataset

train = inpute_missing(train)
test = inpute_missing(test)

In [324]:
def nlp_transforms(dataset):
    """ 
    NLP tranforms here. Default, None...
    """
    banned = ['brindle',
    'tabby',
    'merle',
    'torbie',
    'point',
    'smoke',
    'tiger',
    'tick'
    ]
    
    # Colors
    dataset['color'] = dataset['color'].str.lower()
    dataset['color'] = np.where(dataset['color'] == 'Tortie', 'Torbie', dataset['color'])
    
    dataset['color_type'] = None
    dataset['color_type'] = np.where(dataset['color'].str.contains('brindle'), "brindle", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('tabby'), "tabby", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('merle'), "merle", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('torbie'), "torbie", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('point'), "point", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('smoke'), "smoke", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('tiger'), "tiger", dataset['color_type'])
    dataset['color_type'] = np.where(dataset['color'].str.contains('tick'), "tick", dataset['color_type'])
    
    dataset['num_colors'] = dataset['color'].str.split('/').str.len()
    dataset['color_one'] = dataset['color'].apply(lambda x: x.split('/')[0])
    dataset['color_two'] = dataset['color'].apply(lambda x: x.split('/')[1] if len(x.split('/')) > 1 else 'None')

    dataset['color_one'] = dataset['color_one'].str.replace('/',' ')
    dataset['color_one'] = [' '.join([item for item in x.split() 
                       if item not in banned]) 
                       for x in dataset["color_one"]]

    dataset['color_two'] = dataset['color_two'].str.replace('/',' ')
    dataset['color_two'] = [' '.join([item for item in x.split() 
                       if item not in banned]) 
                       for x in dataset["color_two"]]\
    
    dataset['num_colors'] = np.where((dataset['color_one'] == 'tricolor') | (dataset['color_two'] == 'tricolor'), 3, dataset['num_colors'])
    
    # Breeds
    dataset['breed'] = dataset['breed'].str.lower()
    
    dataset['breed_one'] = dataset['breed'].apply(lambda x: x.split('/')[0])
    dataset['breed_two'] = dataset['breed'].apply(lambda x: x.split('/')[1] if len(x.split('/')) ==2 else 'None')
    dataset['breed_three'] = dataset['breed'].apply(lambda x: x.split('/')[2] if len(x.split('/')) ==3 else 'None')
    
    dataset['num_breed'] = dataset['breed'].str.split('/').str.len()
    dataset['mixed'] = np.where(dataset['breed'].str.contains('mix'), 1,0)
    dataset['mixed'] = np.where(dataset['num_breed'] > 1, 1 ,dataset['mixed'])
    return dataset

train = nlp_transforms(train)
test = nlp_transforms(test)

In [325]:
def computation_transforms(dataset):
    """ 
    New Feature Engineering. Default, None...
    """
    
    # Date stuff
    dataset['datetime'] = pd.to_datetime(dataset['datetime'])
    dataset['year'] = dataset['datetime'].dt.year
    dataset['month'] = dataset['datetime'].dt.month
    dataset['day'] = dataset['datetime'].dt.day
    dataset['hour'] = dataset['datetime'].dt.hour
    dataset['weekday'] = dataset['datetime'].dt.day_name()
    
    #age
    dataset['date_of_birth'] = pd.to_datetime(dataset['date_of_birth'])
    dataset['age'] = (dataset['datetime'] - dataset['date_of_birth']).dt.days
    dataset['birth_year'] = dataset['date_of_birth'].dt.year
    dataset['birth_month'] = dataset['date_of_birth'].dt.month
    # color??
    return dataset

train = computation_transforms(train)
test = computation_transforms(test)

# Train

In [326]:
train.head(1)

,id,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,name,outcome_type,sex,...,num_breed,mixed,year,month,day,hour,weekday,age,birth_year,birth_month
0,1265,2 years,Cat,russian blue mix,blue,2014-04-21,2016-08-05 14:15:00,Smokey,0,Male,...,1,1,2016,8,5,14,Friday,837,2014,4


In [327]:
train.columns

Index(['id', 'age_upon_outcome', 'animal_type', 'breed', 'color',
       'date_of_birth', 'datetime', 'name', 'outcome_type', 'sex',
       'spay_neuter', 'color_type', 'num_colors', 'color_one', 'color_two',
       'breed_one', 'breed_two', 'breed_three', 'num_breed', 'mixed', 'year',
       'month', 'day', 'hour', 'weekday', 'age', 'birth_year', 'birth_month'],
      dtype='object')

In [328]:
train.shape

(54408, 28)

In [329]:
to_drop = ['id','name','age_upon_outcome','datetime','date_of_birth','breed','color']
train = train.drop(to_drop,axis=1)
test = test.drop(to_drop,axis=1)

In [330]:
X_train, X_test = train_test_split(train,test_size=0.2, random_state=42)

train_tf = tfdf.keras.pd_dataframe_to_tf_dataset(X_train, label=label)
test_tf = tfdf.keras.pd_dataframe_to_tf_dataset(X_test, label=label)
predictions = tfdf.keras.pd_dataframe_to_tf_dataset(test)

In [ ]:
# A more complex, but possibly, more accurate model.
models = {
    'rf_baselines': tfdf.keras.RandomForestModel(),
    'rf_tune1': tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1"),
    'gbt_baselines': tfdf.keras.GradientBoostedTreesModel(),
    'gbt_tune1': tfdf.keras.GradientBoostedTreesModel()
    }

evaluation = {}
for key in models:
    print(key)
    #Evaluate: metric logsloss: BinaryCrossentropy
    models[key].compile(metrics=["accuracy"])
    
    #with sys_pipes():
    models[key].fit(x=train_tf)
    evaluation[key] = models[key].evaluate(test_tf, return_dict=True)

rf_baselines
171/171 [==============================] - 2s 10ms/step - loss: 0.0000e+00 - accuracy: 0.8243
rf_tune1
681/681 [==============================] - 1s 1ms/step


In [195]:
evaluation

{'rf_baselines': {'loss': 0.0, 'accuracy': 0.8247565031051636},
 'rf_tune1': {'loss': 0.0, 'accuracy': 0.8263186812400818},
 'gbt_baselines': {'loss': 0.0, 'accuracy': 0.8265025019645691},
 'gbt_tune1': {'loss': 0.0, 'accuracy': 0.8265025019645691}}

In [169]:
evaluation

{'gbt_tune1': {'loss': 0.0, 'accuracy': 0.8293512463569641}}

# Train on Entire Dataset


In [302]:
train_tf = tfdf.keras.pd_dataframe_to_tf_dataset(train, label=label)
# A more complex, but possibly, more accurate model.
models = {
    #'rf_baselines': tfdf.keras.RandomForestModel(),
    #'rf_tune1': tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1", task = tfdf.keras.Task.REGRESSION),
    #'gbt_baselines': tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION),
    'gbt_tune1': tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1")
    }

models['gbt_tune1'].compile(metrics=["accuracy"])
models['gbt_tune1'].fit(x=train_tf)

851/851 [==============================] - 1s 1ms/step


# Predictions

In [303]:
models

{'gbt_tune1': <tensorflow_decision_forests.keras.GradientBoostedTreesModel at 0x7fe71c57cc40>}

In [304]:
scores = models['gbt_tune1'].predict(predictions)

In [305]:
#Label classes: ['adoption', 'no outcome', 'transfer']

In [306]:
scores

array([[8.7180000e-04, 9.9736279e-01, 1.7653710e-03],
       [2.0946354e-02, 7.1948725e-01, 2.5956640e-01],
       [9.3595344e-01, 3.3668408e-03, 6.0679663e-02],
       ...,
       [3.3661008e-01, 3.5512988e-02, 6.2787688e-01],
       [9.6797407e-01, 6.9470480e-03, 2.5078943e-02],
       [1.5179572e-01, 8.3186381e-02, 7.6501787e-01]], dtype=float32)

In [307]:
temp = pd.DataFrame(data=scores, columns=["adoption", "no outcome","transfer"])

In [308]:
temp['id'] = submission.id

In [309]:
temp = temp[['id','adoption','no outcome','transfer']]

In [310]:
temp

,id,adoption,no outcome,transfer
0,67473,0.000872,0.997363,0.001765
1,4629,0.020946,0.719487,0.259566
2,13914,0.935953,0.003367,0.060680
3,53131,0.093944,0.219680,0.686376
4,57800,0.585817,0.039921,0.374262
...,...,...,...,...
23312,9347,0.903591,0.038790,0.057620
23313,3781,0.004891,0.022325,0.972784
23314,74693,0.336610,0.035513,0.627877
23315,44987,0.967974,0.006947,0.025079


In [311]:
temp['no outcome'] = np.where(temp['no outcome'] > .96, 1, temp['no outcome'])
temp['adoption'] = np.where(temp['no outcome'] == 1, 0, temp['adoption'])
temp['transfer'] = np.where(temp['no outcome'] == 1, 0, temp['transfer'])

In [312]:
temp['adoption'] = np.where(temp['adoption'] > .96, 1, temp['adoption'])
temp['no outcome'] = np.where(temp['adoption'] == 1, 0, temp['no outcome'])
temp['transfer'] = np.where(temp['adoption'] == 1, 0, temp['transfer'])

In [313]:
temp['transfer'] = np.where(temp['transfer'] > .96, 1, temp['transfer'])
temp['adoption'] = np.where(temp['transfer'] == 1, 0, temp['adoption'])
temp['no outcome'] = np.where(temp['transfer'] == 1, 0, temp['no outcome'])

In [314]:
temp.to_csv('submission.csv', index=False)

In [316]:
!kaggle competitions submit -c sliced-s01e10-playoffs-2 -f submission.csv -m "Best With Names with outcome updates"

100%|█████████████████████████████████████████| 779k/779k [00:00<00:00, 850kB/s]
400 - Bad Request
